# Import everything

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from torchsummary import summary
import torchvision
import torchvision.transforms as transforms

import os
import argparse

from tqdm.auto import tqdm, trange

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Download and Prepare Data

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


# Define Network

In [4]:
from model import ResNet18
net = ResNet18()

In [5]:
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [6]:
params = summary(net)

Layer (type:depth-idx)                   Param #
├─ResNet: 1-1                            --
|    └─Conv2d: 2-1                       1,728
|    └─BatchNorm2d: 2-2                  128
|    └─Sequential: 2-3                   --
|    |    └─BasicBlock: 3-1              73,984
|    |    └─BasicBlock: 3-2              73,984
|    └─Sequential: 2-4                   --
|    |    └─BasicBlock: 3-3              230,144
|    |    └─BasicBlock: 3-4              295,424
|    └─Sequential: 2-5                   --
|    |    └─BasicBlock: 3-5              919,040
|    |    └─BasicBlock: 3-6              1,180,672
|    └─Sequential: 2-6                   --
|    |    └─BasicBlock: 3-7              3,673,088
|    |    └─BasicBlock: 3-8              4,720,640
|    └─Linear: 2-7                       5,130
Total params: 11,173,962
Trainable params: 11,173,962
Non-trainable params: 0


In [7]:
print(f"Total Parameters: {params.total_params}, Trainable Parameter: {params.trainable_params}")
print(f"Satisfies the assignment criteria: {params.total_params <= 5000000}")

Total Parameters: 11173962, Trainable Parameter: 11173962
Satisfies the assignment criteria: False


# Hyperparameters

In [8]:
lr = 0.1

In [9]:
best_acc = 0

# Helper Functions

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [11]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(total=len(trainloader), position=1)
    pbar.set_description('Train Epoch %i' % epoch)
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        loss = train_loss/(batch_idx+1)
        acc = 100.*correct/total
        
        pbar.set_postfix(loss=loss, acc=acc)
        pbar.update()

    pbar.close()

In [12]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        
        pbar = tqdm(total=len(testloader), position=1)
        pbar.set_description('Test Epoch %i' % epoch)
        
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            loss = test_loss/(batch_idx+1)
            acc = 100.*correct/total

            pbar.set_postfix(loss=loss, acc=acc)
            pbar.update()
        
        pbar.close()

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

# Train Network

In [13]:
start_epoch = 0
total_epochs = 200

In [ ]:
for epoch in trange(start_epoch, total_epochs, desc='Epoch'):
    train(epoch)
    test(epoch)
    scheduler.step()

# Results